# Titanic - GroupKFold版 (最終最適化)

## 🎯 改善点:
- ✅ **GroupKFold**: Ticketでグループ化してデータリーク防止
- ✅ **Is_Married修正**: Title統合前に判定
- ✅ **最適閾値探索**: OOFで最適化
- ✅ **重み最適化**: ロジスティック回帰でメタ学習
- ✅ **CatBoost最適化**: 文字列カテゴリを直接使用
- ✅ **数値特徴量追加**: Age, Fareを連続値のまま使用

## 1. セットアップとデータ読み込み

In [1]:
# ライブラリのインポート
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import warnings
import string
import pickle
from typing import Dict, List, Tuple
warnings.filterwarnings('ignore')

from sklearn.model_selection import GroupKFold  # ✅ GroupKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, log_loss
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

from catboost import CatBoostClassifier, Pool
from xgboost import XGBClassifier

# 定数設定
RANDOM_STATE = 42
N_FOLDS = 5

# 乱数シード固定
np.random.seed(RANDOM_STATE)

print("ライブラリのインポート完了")

ライブラリのインポート完了


In [2]:
# データ読み込み
df_train = pd.read_csv('../input/train.csv')
df_test = pd.read_csv('../input/test.csv')

print(f"Train shape: {df_train.shape}")
print(f"Test shape: {df_test.shape}")
print(f"\nTarget distribution:")
print(df_train['Perished'].value_counts(normalize=True))

df_train.head()

Train shape: (891, 12)
Test shape: (418, 11)

Target distribution:
Perished
1    0.616162
0    0.383838
Name: proportion, dtype: float64


,PassengerId,Perished,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,0,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,1,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## 2. 特徴量エンジニアリング

In [3]:
def extract_surname(data):
    """名前から姓を抽出"""
    families = []
    
    for i in range(len(data)):
        name = data.iloc[i]
        if '(' in name:
            name_no_bracket = name.split('(')[0]
        else:
            name_no_bracket = name
        
        family = name_no_bracket.split(',')[0]
        
        for c in string.punctuation:
            family = family.replace(c, '').strip()
        
        families.append(family)
    
    return families


def create_features(df_train, df_test):
    """
    特徴量エンジニアリングを実行（GroupKFold対応版）
    """
    train = df_train.copy()
    test = df_test.copy()
    
    all_data = pd.concat([train, test], axis=0, sort=False).reset_index(drop=True)
    train_idx = ~all_data['Perished'].isna()
    
    print("特徴量エンジニアリング開始...")
    print("🎯 GroupKFold対応版\n")
    
    # 1. Title（敬称）抽出 - 生のTitleを先に保存
    all_data['Title_raw'] = all_data['Name'].str.split(', ', expand=True)[1].str.split('.', expand=True)[0]
    
    # 2. 結婚フラグ - 生のTitleで判定
    all_data['Is_Married'] = (all_data['Title_raw'] == 'Mrs').astype(int)
    
    # 3. Title 正規化
    all_data['Title'] = all_data['Title_raw'].replace(
        ['Miss', 'Mrs', 'Ms', 'Mlle', 'Lady', 'Mme', 'the Countess', 'Dona'], 'Miss/Mrs/Ms'
    )
    all_data['Title'] = all_data['Title'].replace(
        ['Dr', 'Col', 'Major', 'Jonkheer', 'Capt', 'Sir', 'Don', 'Rev'], 'Dr/Military/Noble/Clergy'
    )
    
    # 4. Family（姓）
    all_data['Family'] = extract_surname(all_data['Name'])
    
    # 5. Family_Size
    all_data['Family_Size'] = all_data['SibSp'] + all_data['Parch'] + 1
    
    # 6. Family_Size_Grouped
    family_map = {
        1: 'Alone',
        2: 'Small', 3: 'Small', 4: 'Small',
        5: 'Medium', 6: 'Medium',
        7: 'Large', 8: 'Large', 11: 'Large'
    }
    all_data['Family_Size_Grouped'] = all_data['Family_Size'].map(family_map)
    
    # 7. Ticket_Frequency - 訓練データのみで計算
    train_ticket_freq = all_data.loc[train_idx].groupby('Ticket')['Ticket'].transform('count')
    train_ticket_map = all_data.loc[train_idx].groupby('Ticket').size().to_dict()
    
    all_data.loc[train_idx, 'Ticket_Frequency'] = train_ticket_freq
    all_data.loc[~train_idx, 'Ticket_Frequency'] = all_data.loc[~train_idx, 'Ticket'].map(train_ticket_map).fillna(1)
    
    # 8. Deck
    all_data['Deck'] = all_data['Cabin'].apply(lambda s: s[0] if pd.notnull(s) else 'M')
    all_data['Deck'] = all_data['Deck'].replace(['A', 'B', 'C'], 'ABC')
    all_data['Deck'] = all_data['Deck'].replace(['D', 'E'], 'DE')
    all_data['Deck'] = all_data['Deck'].replace(['F', 'G'], 'FG')
    all_data['Deck'] = all_data['Deck'].replace(['T'], 'M')
    
    # 9. Age補完（連続値も保持）
    all_data['Age'] = pd.to_numeric(all_data['Age'], errors='coerce')
    age_by_pclass_sex = all_data.loc[train_idx].groupby(['Sex', 'Pclass'])['Age'].median()
    
    for pclass in [1, 2, 3]:
        for sex in ['male', 'female']:
            mask = (all_data['Age'].isnull()) & (all_data['Pclass'] == pclass) & (all_data['Sex'] == sex)
            all_data.loc[mask, 'Age'] = age_by_pclass_sex.loc[(sex, pclass)]
    
    all_data['Age_Band'] = pd.cut(all_data['Age'], bins=[0, 12, 18, 30, 50, 80], 
                                    labels=['Child', 'Teen', 'Adult', 'Middle', 'Senior'])
    
    # 10. Fare補完（連続値も保持）
    train_fare_median = all_data.loc[train_idx, 'Fare'].median()
    all_data['Fare'] = all_data['Fare'].fillna(train_fare_median)
    
    train_fare_quantiles = all_data.loc[train_idx, 'Fare'].quantile([0.25, 0.5, 0.75]).values
    all_data['Fare_Band'] = pd.cut(
        all_data['Fare'],
        bins=[-np.inf] + train_fare_quantiles.tolist() + [np.inf],
        labels=['Low', 'Medium', 'High', 'Very_High']
    )
    
    # 11. Embarked補完
    all_data['Embarked'] = all_data['Embarked'].fillna('S')
    
    # 12. Sex x Pclass 交互作用
    all_data['Sex_Pclass'] = all_data['Sex'] + '_' + all_data['Pclass'].astype(str)
    
    print(f"✅ 特徴量作成完了: {all_data.shape[1]} columns")
    print(f"✅ Is_Married有効数: {all_data['Is_Married'].sum()}件")
    
    # ✅ GroupKFold用にTicketを保持
    print(f"✅ ユニークTicket数: {all_data.loc[train_idx, 'Ticket'].nunique()}件")
    
    train_processed = all_data[train_idx].reset_index(drop=True)
    test_processed = all_data[~train_idx].reset_index(drop=True)
    
    return train_processed, test_processed


train_df, test_df = create_features(df_train, df_test)

print(f"\nTrain shape: {train_df.shape}")
print(f"Test shape: {test_df.shape}")

特徴量エンジニアリング開始...
🎯 GroupKFold対応版

✅ 特徴量作成完了: 23 columns
✅ Is_Married有効数: 197件
✅ ユニークTicket数: 681件

Train shape: (891, 23)
Test shape: (418, 23)


In [4]:
def prepare_data(train_df, test_df):
    """
    データを準備（GroupKFold用にTicketも返す）
    """
    feature_cols = [
        'Pclass', 'Age', 'Fare', 'Family_Size', 'Is_Married',
        'Embarked', 'Title', 'Family_Size_Grouped', 'Ticket_Frequency', 'Deck',
        'Age_Band', 'Fare_Band', 'Sex_Pclass'
    ]
    
    train = train_df[feature_cols + ['Perished']].copy()
    test = test_df[feature_cols].copy()
    
    cat_features = ['Embarked', 'Title', 'Family_Size_Grouped', 'Deck', 'Age_Band', 'Fare_Band', 'Sex_Pclass']
    
    # RF/XGB用: LabelEncoder
    train_encoded = train.copy()
    test_encoded = test.copy()
    
    le_dict = {}
    for col in cat_features:
        le = LabelEncoder()
        le.fit(train_encoded[col].astype(str))
        
        train_encoded[col] = le.transform(train_encoded[col].astype(str))
        
        test_col_str = test_encoded[col].astype(str)
        unknown_mask = ~test_col_str.isin(le.classes_)
        
        if unknown_mask.any():
            most_frequent = train_df[col].mode()[0]
            test_encoded.loc[unknown_mask, col] = most_frequent
        
        test_encoded[col] = le.transform(test_encoded[col].astype(str))
        le_dict[col] = le
    
    X_encoded = train_encoded.drop('Perished', axis=1)
    y = train_encoded['Perished']
    X_test_encoded = test_encoded
    
    # CatBoost用: 文字列のまま
    X_catboost = train.drop('Perished', axis=1)
    X_test_catboost = test
    
    # ✅ GroupKFold用のグループ変数
    groups = train_df['Ticket']
    
    print(f"\nX_encoded shape: {X_encoded.shape}")
    print(f"y shape: {y.shape}")
    print(f"groups shape: {groups.shape}")
    print(f"✅ GroupKFold用にTicketグループを保持")
    
    return X_encoded, X_catboost, y, X_test_encoded, X_test_catboost, cat_features, groups


X_encoded, X_catboost, y, X_test_encoded, X_test_catboost, cat_features, groups = prepare_data(train_df, test_df)
print(f"\nCategorical features: {cat_features}")


X_encoded shape: (891, 13)
y shape: (891,)
groups shape: (891,)
✅ GroupKFold用にTicketグループを保持

Categorical features: ['Embarked', 'Title', 'Family_Size_Grouped', 'Deck', 'Age_Band', 'Fare_Band', 'Sex_Pclass']


## 3. ハイパーパラメータ設定

In [5]:
# RandomForest
best_params_randomforest = {
    'n_estimators': 1200,
    'max_depth': 6,
    'min_samples_split': 6,
    'min_samples_leaf': 3,
    'max_features': 'sqrt',
    'bootstrap': True,
    'max_samples': 0.9,
    'criterion': 'gini',
    'class_weight': None,
    'random_state': RANDOM_STATE,
    'n_jobs': -1
}

# CatBoost
best_params_catboost = {
    'iterations': 1000,
    'learning_rate': 0.03,
    'depth': 6,
    'l2_leaf_reg': 6.0,
    'bagging_temperature': 0.5,
    'random_strength': 1.0,
    'rsm': 0.9,
    'loss_function': 'Logloss',
    'eval_metric': 'Logloss',
    'random_state': RANDOM_STATE,
    'verbose': 0
}

# XGBoost
best_params_xgboost = {
    'n_estimators': 500,
    'learning_rate': 0.05,
    'max_depth': 6,
    'min_child_weight': 3,
    'gamma': 0.1,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'reg_alpha': 0.1,
    'reg_lambda': 1.0,
    'random_state': RANDOM_STATE,
    'eval_metric': 'logloss',
    'tree_method': 'hist'
}

print("ハイパーパラメータ設定完了")

ハイパーパラメータ設定完了


## 4. GroupKFold学習

In [6]:
def train_with_group_kfold(X_encoded, X_catboost, y, X_test_encoded, X_test_catboost, cat_features, groups, n_folds=5):
    """
    GroupKFoldで学習（Ticketでグループ化）
    """
    gkf = GroupKFold(n_splits=n_folds)  # ✅ GroupKFold
    
    # 確率を保存
    oof_proba_rf = np.zeros(len(X_encoded))
    oof_proba_cat = np.zeros(len(X_encoded))
    oof_proba_xgb = np.zeros(len(X_encoded))
    
    test_proba_rf = np.zeros((len(X_test_encoded), n_folds))
    test_proba_cat = np.zeros((len(X_test_encoded), n_folds))
    test_proba_xgb = np.zeros((len(X_test_encoded), n_folds))
    
    models_rf = []
    models_cat = []
    models_xgb = []
    
    train_scores = {'rf': [], 'cat': [], 'xgb': []}
    valid_scores = {'rf': [], 'cat': [], 'xgb': []}
    fold_sizes = []
    
    print("="*60)
    print("🎯 GroupKFold Training (grouped by Ticket)")
    print("="*60)
    
    for fold, (train_idx, valid_idx) in enumerate(gkf.split(X_encoded, y, groups=groups)):
        print(f"\n{'='*60}")
        print(f"Fold {fold + 1}/{n_folds}")
        print(f"  Train: {len(train_idx)} samples, Valid: {len(valid_idx)} samples")
        fold_sizes.append((len(train_idx), len(valid_idx)))
        print(f"{'='*60}")
        
        # RF/XGB用
        X_train_enc, X_valid_enc = X_encoded.iloc[train_idx], X_encoded.iloc[valid_idx]
        y_train, y_valid = y.iloc[train_idx], y.iloc[valid_idx]
        
        # CatBoost用
        X_train_cat, X_valid_cat = X_catboost.iloc[train_idx], X_catboost.iloc[valid_idx]
        
        # RandomForest
        print("\n[1/3] Training RandomForest...")
        model_rf = RandomForestClassifier(**best_params_randomforest)
        model_rf.fit(X_train_enc, y_train)
        
        oof_proba_rf[valid_idx] = model_rf.predict_proba(X_valid_enc)[:, 1]
        test_proba_rf[:, fold] = model_rf.predict_proba(X_test_encoded)[:, 1]
        models_rf.append(model_rf)
        
        valid_acc_rf = accuracy_score(y_valid, model_rf.predict(X_valid_enc))
        train_acc_rf = accuracy_score(y_train, model_rf.predict(X_train_enc))
        train_scores['rf'].append(train_acc_rf)
        valid_scores['rf'].append(valid_acc_rf)
        
        print(f"  Train: {train_acc_rf:.4f} | Valid: {valid_acc_rf:.4f} | Gap: {train_acc_rf - valid_acc_rf:.4f}")
        
        # CatBoost
        print("\n[2/3] Training CatBoost...")
        train_pool = Pool(X_train_cat, y_train, cat_features=cat_features)
        valid_pool = Pool(X_valid_cat, y_valid, cat_features=cat_features)
        test_pool = Pool(X_test_catboost, cat_features=cat_features)
        
        model_cat = CatBoostClassifier(**best_params_catboost)
        model_cat.fit(train_pool, eval_set=valid_pool, early_stopping_rounds=50, verbose=0)
        
        oof_proba_cat[valid_idx] = model_cat.predict_proba(valid_pool)[:, 1]
        test_proba_cat[:, fold] = model_cat.predict_proba(test_pool)[:, 1]
        models_cat.append(model_cat)
        
        valid_acc_cat = accuracy_score(y_valid, model_cat.predict(valid_pool))
        train_acc_cat = accuracy_score(y_train, model_cat.predict(train_pool))
        train_scores['cat'].append(train_acc_cat)
        valid_scores['cat'].append(valid_acc_cat)
        
        print(f"  Train: {train_acc_cat:.4f} | Valid: {valid_acc_cat:.4f} | Gap: {train_acc_cat - valid_acc_cat:.4f}")
        
        # XGBoost
        print("\n[3/3] Training XGBoost...")
        model_xgb = XGBClassifier(**best_params_xgboost)
        model_xgb.fit(
            X_train_enc, y_train, 
            eval_set=[(X_train_enc, y_train), (X_valid_enc, y_valid)],
            verbose=0,
            early_stopping_rounds=50
        )
        
        oof_proba_xgb[valid_idx] = model_xgb.predict_proba(X_valid_enc)[:, 1]
        test_proba_xgb[:, fold] = model_xgb.predict_proba(X_test_encoded)[:, 1]
        models_xgb.append(model_xgb)
        
        valid_acc_xgb = accuracy_score(y_valid, model_xgb.predict(X_valid_enc))
        train_acc_xgb = accuracy_score(y_train, model_xgb.predict(X_train_enc))
        train_scores['xgb'].append(train_acc_xgb)
        valid_scores['xgb'].append(valid_acc_xgb)
        
        print(f"  Train: {train_acc_xgb:.4f} | Valid: {valid_acc_xgb:.4f} | Gap: {train_acc_xgb - valid_acc_xgb:.4f}")
    
    # CV結果サマリー
    print(f"\n{'='*60}")
    print("Cross-Validation Results (GroupKFold)")
    print(f"{'='*60}")
    
    print(f"\nFold sizes (Train, Valid):")
    for i, (tr, va) in enumerate(fold_sizes):
        print(f"  Fold {i+1}: ({tr}, {va})")
    
    print(f"\nModel Performance:")
    for name, key in [('RandomForest', 'rf'), ('CatBoost', 'cat'), ('XGBoost', 'xgb')]:
        train_mean = np.mean(train_scores[key])
        train_std = np.std(train_scores[key])
        valid_mean = np.mean(valid_scores[key])
        valid_std = np.std(valid_scores[key])
        gap = train_mean - valid_mean
        
        print(f"{name}:")
        print(f"  Train: {train_mean:.4f} ± {train_std:.4f}")
        print(f"  Valid: {valid_mean:.4f} ± {valid_std:.4f}")
        print(f"  Gap:   {gap:.4f}")
    
    return {
        'oof_proba_rf': oof_proba_rf,
        'oof_proba_cat': oof_proba_cat,
        'oof_proba_xgb': oof_proba_xgb,
        'test_proba_rf': test_proba_rf.mean(axis=1),
        'test_proba_cat': test_proba_cat.mean(axis=1),
        'test_proba_xgb': test_proba_xgb.mean(axis=1),
        'models_rf': models_rf,
        'models_cat': models_cat,
        'models_xgb': models_xgb
    }


# 学習実行
results = train_with_group_kfold(X_encoded, X_catboost, y, X_test_encoded, X_test_catboost, cat_features, groups, n_folds=N_FOLDS)

🎯 GroupKFold Training (grouped by Ticket)

Fold 1/5
  Train: 712 samples, Valid: 179 samples

[1/3] Training RandomForest...
  Train: 0.8834 | Valid: 0.8603 | Gap: 0.0231

[2/3] Training CatBoost...
  Train: 0.8596 | Valid: 0.8603 | Gap: -0.0008

[3/3] Training XGBoost...
  Train: 0.8904 | Valid: 0.8436 | Gap: 0.0469

Fold 2/5
  Train: 713 samples, Valid: 178 samples

[1/3] Training RandomForest...
  Train: 0.8780 | Valid: 0.8371 | Gap: 0.0409

[2/3] Training CatBoost...
  Train: 0.8541 | Valid: 0.8090 | Gap: 0.0451

[3/3] Training XGBoost...
  Train: 0.8850 | Valid: 0.8315 | Gap: 0.0535

Fold 3/5
  Train: 713 samples, Valid: 178 samples

[1/3] Training RandomForest...
  Train: 0.8724 | Valid: 0.8146 | Gap: 0.0578

[2/3] Training CatBoost...
  Train: 0.8555 | Valid: 0.8202 | Gap: 0.0353

[3/3] Training XGBoost...
  Train: 0.8962 | Valid: 0.8202 | Gap: 0.0760

Fold 4/5
  Train: 713 samples, Valid: 178 samples

[1/3] Training RandomForest...
  Train: 0.8668 | Valid: 0.8371 | Gap: 0.0297


## 5. アンサンブル最適化

In [7]:
# 最適閾値探索
print("="*60)
print("🎯 最適閾値探索")
print("="*60)

ensemble_proba = (
    results['oof_proba_rf'] +
    results['oof_proba_cat'] +
    results['oof_proba_xgb']
) / 3

thresholds = np.linspace(0.3, 0.7, 81)
best_th, best_acc = 0.5, 0.0

for th in thresholds:
    acc = accuracy_score(y, (ensemble_proba >= th).astype(int))
    if acc > best_acc:
        best_acc, best_th = acc, th

print(f"\n最適閾値: {best_th:.3f}")
print(f"OOF Accuracy: {best_acc:.4f}")
print(f"デフォルト(0.5): {accuracy_score(y, (ensemble_proba >= 0.5).astype(int)):.4f}")

🎯 最適閾値探索

最適閾値: 0.490
OOF Accuracy: 0.8305
デフォルト(0.5): 0.8272


In [8]:
# メタ学習
print("\n" + "="*60)
print("🎯 メタ学習 (重み最適化)")
print("="*60)

oof_meta = np.vstack([
    results['oof_proba_rf'],
    results['oof_proba_cat'],
    results['oof_proba_xgb'],
]).T

test_meta = np.vstack([
    results['test_proba_rf'],
    results['test_proba_cat'],
    results['test_proba_xgb'],
]).T

meta_model = LogisticRegression(max_iter=1000, random_state=RANDOM_STATE)
meta_model.fit(oof_meta, y)

ensemble_meta_proba = meta_model.predict_proba(oof_meta)[:, 1]
meta_oof_acc = accuracy_score(y, (ensemble_meta_proba >= 0.5).astype(int))

print(f"\nメタモデル重み:")
print(f"  RF:  {meta_model.coef_[0][0]:.4f}")
print(f"  Cat: {meta_model.coef_[0][1]:.4f}")
print(f"  XGB: {meta_model.coef_[0][2]:.4f}")
print(f"\nメタモデル OOF: {meta_oof_acc:.4f}")
print(f"単純平均 OOF: {best_acc:.4f}")


🎯 メタ学習 (重み最適化)

メタモデル重み:
  RF:  1.4657
  Cat: 1.8140
  XGB: 2.2902

メタモデル OOF: 0.8305
単純平均 OOF: 0.8305


In [9]:
# 最終結果
print("\n" + "="*60)
print("📊 最終結果サマリー (GroupKFold)")
print("="*60)

print(f"\n個別モデル:")
print(f"  RF:  {accuracy_score(y, (results['oof_proba_rf'] >= 0.5).astype(int)):.4f}")
print(f"  Cat: {accuracy_score(y, (results['oof_proba_cat'] >= 0.5).astype(int)):.4f}")
print(f"  XGB: {accuracy_score(y, (results['oof_proba_xgb'] >= 0.5).astype(int)):.4f}")

print(f"\nアンサンブル:")
print(f"  単純平均 (閾値0.5): {accuracy_score(y, (ensemble_proba >= 0.5).astype(int)):.4f}")
print(f"  単純平均 (最適閾値): {best_acc:.4f}")
print(f"  メタ学習: {meta_oof_acc:.4f}")

if meta_oof_acc >= best_acc:
    print(f"\n✅ 採用: メタ学習")
    final_method = 'meta'
    final_oof_score = meta_oof_acc
else:
    print(f"\n✅ 採用: 単純平均+最適閾値")
    final_method = 'threshold'
    final_oof_score = best_acc


📊 最終結果サマリー (GroupKFold)

個別モデル:
  RF:  0.8316
  Cat: 0.8249
  XGB: 0.8260

アンサンブル:
  単純平均 (閾値0.5): 0.8272
  単純平均 (最適閾値): 0.8305
  メタ学習: 0.8305

✅ 採用: メタ学習


## 6. テスト予測と提出

In [10]:
# テスト予測
if final_method == 'meta':
    test_ensemble_proba = meta_model.predict_proba(test_meta)[:, 1]
    final_predictions = (test_ensemble_proba >= 0.5).astype(int)
    output_path = '../output/submission_group_kfold_meta.csv'
else:
    test_ensemble_proba = (
        results['test_proba_rf'] +
        results['test_proba_cat'] +
        results['test_proba_xgb']
    ) / 3
    final_predictions = (test_ensemble_proba >= best_th).astype(int)
    output_path = '../output/submission_group_kfold_threshold.csv'

submission = pd.DataFrame({
    'PassengerId': test_df['PassengerId'],
    'Perished': final_predictions
})

os.makedirs('../output', exist_ok=True)
submission.to_csv(output_path, index=False)

print(f"\n提出ファイル: {output_path}")
print(f"\n予測分布:")
print(f"  Survived (0): {(final_predictions == 0).sum()} ({(final_predictions == 0).sum() / len(final_predictions) * 100:.1f}%)")
print(f"  Perished (1): {(final_predictions == 1).sum()} ({(final_predictions == 1).sum() / len(final_predictions) * 100:.1f}%)")

print(f"\n最初の10行:")
print(submission.head(10))


提出ファイル: ../output/submission_group_kfold_meta.csv

予測分布:
  Survived (0): 145 (34.7%)
  Perished (1): 273 (65.3%)

最初の10行:
   PassengerId  Perished
0          892         1
1          893         1
2          894         1
3          895         1
4          896         1
5          897         1
6          898         0
7          899         1
8          900         0
9          901         1


In [11]:
print("\n" + "="*60)
print("✅ すべての処理が完了しました!")
print("="*60)
print(f"\n最終OOFスコア: {final_oof_score:.4f}")
print(f"手法: {final_method}")
print(f"\n🎯 実装済み最適化:")
print(f"  ✅ GroupKFold (Ticketでグループ化)")
print(f"  ✅ Is_Married修正")
print(f"  ✅ 最適閾値探索")
print(f"  ✅ メタ学習")
print(f"  ✅ CatBoost最適化")
print(f"  ✅ 数値特徴量追加")
print(f"\n📊 GroupKFoldによりデータリーク防止")
print(f"📊 より現実的なOOFスコアを取得")


✅ すべての処理が完了しました!

最終OOFスコア: 0.8305
手法: meta

🎯 実装済み最適化:
  ✅ GroupKFold (Ticketでグループ化)
  ✅ Is_Married修正
  ✅ 最適閾値探索
  ✅ メタ学習
  ✅ CatBoost最適化
  ✅ 数値特徴量追加

📊 GroupKFoldによりデータリーク防止
📊 より現実的なOOFスコアを取得
